# Hands-On Pertemuan 6: Data Processing dengan Apache Spark

## Tujuan:
- Memahami dan mempraktikkan data processing menggunakan Apache Spark.
- Menggunakan Spark untuk operasi data yang efisien pada dataset besar.
- Menerapkan teknik canggih dalam Spark untuk mengatasi kasus penggunaan nyata.

### 1. Pengenalan Spark DataFrames
Spark DataFrame menyediakan struktur data yang optimal dengan operasi yang dioptimalkan untuk pemrosesan data besar, yang sangat mirip dengan DataFrame di Pandas atau di RDBMS.

- **Tugas 1**: Buat DataFrame sederhana di Spark dan eksplorasi beberapa fungsi dasar yang tersedia.

In [6]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=172182ba13f60c499ca176e877596b18105fb709dcfb0383d4e3bde03e6236b0
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [7]:
# Contoh membuat DataFrame sederhana dan operasi dasar

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HandsOnPertemuan6').getOrCreate()

data = [('James', 'Sales', 3000),
        ('Michael', 'Sales', 4600),
        ('Robert', 'Sales', 4100),
        ('Maria', 'Finance', 3000)]
columns = ['EmployeeName', 'Department', 'Salary']

df = spark.createDataFrame(data, schema=columns)
df.show()


+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|       James|     Sales|  3000|
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
|       Maria|   Finance|  3000|
+------------+----------+------+



### 2. Transformasi Dasar dengan DataFrames
Pemrosesan data meliputi transformasi seperti filtering, selections, dan aggregations. Spark menyediakan cara efisien untuk melaksanakan operasi ini.

- **Tugas 2**: Gunakan operasi filter, select, groupBy untuk mengekstrak informasi dari data, serta lakukan agregasi data untuk mendapatkan insight tentang dataset menggunakan perintah seperti mean, max, sum.

In [16]:
# Contoh operasi transformasi DataFrame
from pyspark.sql.functions import col, mean, max, sum

df.select('EmployeeName', 'Salary').show()
df.filter(col('Salary') > 3000).show()
df.groupBy('Department').avg('Salary').show()

aggregated_df = df.groupBy("Department").agg(
    mean("Salary").alias("Average_Salary"),
    max("Salary").alias("Max_Salary"),
    sum("Salary").alias("Total_Salary")
)
print("Aggregated Data (grouped by Department):")
aggregated_df.show()

+------------+------+
|EmployeeName|Salary|
+------------+------+
|       James|  3000|
|     Michael|  4600|
|      Robert|  4100|
|       Maria|  3000|
+------------+------+

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
+------------+----------+------+

+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|     Sales|     3900.0|
|   Finance|     3000.0|
+----------+-----------+

Aggregated Data (grouped by Department):
+----------+--------------+----------+------------+
|Department|Average_Salary|Max_Salary|Total_Salary|
+----------+--------------+----------+------------+
|     Sales|        3900.0|      4600|       11700|
|   Finance|        3000.0|      3000|        3000|
+----------+--------------+----------+------------+



### 3. Bekerja dengan Tipe Data Kompleks
Spark mendukung tipe data yang kompleks seperti maps, arrays, dan structs yang memungkinkan operasi yang lebih kompleks pada dataset yang kompleks.

- **Tugas 3**: Eksplorasi bagaimana mengolah tipe data kompleks dalam Spark DataFrames.

In [18]:
# Create SalaryBonus as 10% of Salary
df = df.withColumn('SalaryBonus', df['Salary'] * 0.1)

# Now you can calculate TotalCompensation
df.withColumn('TotalCompensation', df['Salary'] + df['SalaryBonus']).show()

df.show()


+------------+----------+------+-----------+-----------------+
|EmployeeName|Department|Salary|SalaryBonus|TotalCompensation|
+------------+----------+------+-----------+-----------------+
|       James|     Sales|  3000|      300.0|           3300.0|
|     Michael|     Sales|  4600|      460.0|           5060.0|
|      Robert|     Sales|  4100|      410.0|           4510.0|
|       Maria|   Finance|  3000|      300.0|           3300.0|
+------------+----------+------+-----------+-----------------+

+------------+----------+------+-----------+
|EmployeeName|Department|Salary|SalaryBonus|
+------------+----------+------+-----------+
|       James|     Sales|  3000|      300.0|
|     Michael|     Sales|  4600|      460.0|
|      Robert|     Sales|  4100|      410.0|
|       Maria|   Finance|  3000|      300.0|
+------------+----------+------+-----------+



### 4. Operasi Data Lanjutan
Menggunakan Spark untuk operasi lanjutan seperti window functions, user-defined functions (UDFs), dan mengoptimalkan query.

- **Tugas 4**: Implementasikan window function untuk menghitung running totals atau rangkings.

In [24]:
# Contoh menggunakan window functions
from pyspark.sql.window import Window
from pyspark.sql import functions as F

windowSpec = Window.partitionBy('Department').orderBy('Salary')
df.withColumn('Rank', F.rank().over(windowSpec)).show()
df.withColumn('Running_Total', F.sum('Salary').over(windowSpec)).show()



+------------+----------+------+-----------+----+
|EmployeeName|Department|Salary|SalaryBonus|Rank|
+------------+----------+------+-----------+----+
|       Maria|   Finance|  3000|      300.0|   1|
|       James|     Sales|  3000|      300.0|   1|
|      Robert|     Sales|  4100|      410.0|   2|
|     Michael|     Sales|  4600|      460.0|   3|
+------------+----------+------+-----------+----+

+------------+----------+------+-----------+-------------+
|EmployeeName|Department|Salary|SalaryBonus|Running_Total|
+------------+----------+------+-----------+-------------+
|       Maria|   Finance|  3000|      300.0|         3000|
|       James|     Sales|  3000|      300.0|         3000|
|      Robert|     Sales|  4100|      410.0|         7100|
|     Michael|     Sales|  4600|      460.0|        11700|
+------------+----------+------+-----------+-------------+



### 5. Kesimpulan dan Eksplorasi Lebih Lanjut
Review apa yang telah dipelajari tentang pemrosesan data menggunakan Spark dan eksplorasi teknik lebih lanjut untuk mengoptimalkan pemrosesan data Anda.
- **Tugas 5**: Buat ringkasan dari semua operasi yang telah dilakukan dan bagaimana teknik ini dapat diterapkan pada proyek data Anda.